In [ ]:
!pip install -q transformers datasets sentencepiece accelerate sacrebleu nltk python-Levenshtein evaluate rouge_score>> None
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from datasets import load_dataset
from IPython.display import display, HTML

# Загрузка CSV
dataset = load_dataset("csv", data_files="literary_corpus_for_spellcheck.csv", split="train")
dataset = dataset.shuffle(seed=42)

html = "<table><tr><th>Пример</th><th>Ошибочный текст</th><th>Исправленный текст</th></tr>"
for i in range(3):
    src = dataset[i]['source']
    tgt = dataset[i]['correction']
    html += f"<tr><td>{i+1}</td><td>{src}</td><td>{tgt}</td></tr>"
html += "</table>"

display(HTML(html))

Пример,Ошибочный текст,Исправленный текст
1,"Мена немного качает,и я чувствую слабость ва всем теле","Меня немного качает, и я чувствую слабость во всем теле."
2,"Пазже он напишет: «Нас мало, нас можит бить читвиро», а в те времена ан был уникален как язык у калокола","Позже он напишет: «Нас мало, нас может быть четверо», а в те времена он был уникален, как язык у колокола."
3,"Чериз час, уже в глубоких сумерках к Дусеприбежали из деревни","Через час, уже в глубоких сумерках, к Дусе прибежали из деревни."


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "ai-forever/sage-fredt5-distilled-95m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Экономим память
model.gradient_checkpointing_enable()
model.config.use_cache = False

In [ ]:
max_length = 128

def preprocess(batch):
    inputs = tokenizer(batch["source"], truncation=True, padding=True, max_length=max_length)
    targets = tokenizer(batch["correction"], truncation=True, padding=True, max_length=max_length)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./sage_finetuned_full",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=500,
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.712800
1000,0.100800
1500,0.067100
2000,0.058800
2500,0.059200
3000,0.051100
3500,0.048200
4000,0.048900
4500,0.046800
5000,0.050000


TrainOutput(global_step=6105, training_loss=0.3560683298852969, metrics={'train_runtime': 1163.3591, 'train_samples_per_second': 20.988, 'train_steps_per_second': 5.248, 'total_flos': 1309808777232384.0, 'train_loss': 0.3560683298852969, 'epoch': 1.0})

In [ ]:
trainer.save_model("sage_finetuned_full")
tokenizer.save_pretrained("sage_finetuned_full")
print("Модель успешно сохранена.")

Модель успешно сохранена.


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sage_finetuned_full")
tokenizer = AutoTokenizer.from_pretrained("sage_finetuned_full")

In [ ]:
import re

def clean_sentence(text):
    match = re.search(r"(.+?[.!?])", text)
    return match.group(1).strip() if match else text.strip()

def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        num_beams=4,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return clean_sentence(raw)

In [ ]:
examples = [
    "Я аабажаю шахматы и хачю выиграть.",
    "Она ушла с уливци нипрощавшись.",
    "Мой брат рисует на стикере и ходит на хокей.",
    "Привет ета тетрадка вазьми пожалуста."
]

# Формируем HTML
html = """
<style>
table {border-collapse: collapse; width: 100%;}
th, td {border: 1px solid #ddd; padding: 8px;}
th {background-color: #f2f2f2;}
</style>
<table>
<tr>
<th>№</th>
<th>Ошибочный текст</th>
<th>Исправленный текст</th>
</tr>
"""

for i, text in enumerate(examples):
    corrected = correct_text(text)
    html += f"<tr><td>{i+1}</td><td>{text}</td><td>{corrected}</td></tr>"

html += "</table>"

# Отображение
display(HTML(html))

№,Ошибочный текст,Исправленный текст
1,Я аабажаю шахматы и хачю выиграть.,Я обожаю шахматы и хочу выиграть.
2,Она ушла с уливци нипрощавшись.,"Она ушла с улицы, непрощавшись."
3,Мой брат рисует на стикере и ходит на хокей.,Мой брат рисует на стикере и ходит на хокей.
4,Привет ета тетрадка вазьми пожалуста.,"Привет, это тетрадка, возьми, пожалуйста."


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Оригинальный FRED-T5
orig_model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/FRED-T5-large-spell")
orig_tokenizer = AutoTokenizer.from_pretrained("ai-forever/FRED-T5-large-spell")

# ByT5
byt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small")
byt5_tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [ ]:
import requests

def correct_with_speller(text):
    url = "https://speller.yandex.net/services/spellservice.json/checkText"
    params = {"text": text, "lang": "ru"}
    response = requests.get(url, params=params).json()

    corrected = text
    offset = 0
    for err in response:
        word = err['word']
        suggestions = err['s']
        if suggestions:
            start = err['pos'] + offset
            end = start + len(word)
            replacement = suggestions[0]
            corrected = corrected[:start] + replacement + corrected[end:]
            offset += len(replacement) - len(word)
    return corrected

In [ ]:
import re

def clean_sentence(text):
    match = re.search(r"(.+?[.!?])", text)
    return match.group(1).strip() if match else text.strip()

def generate_finetuned(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,  # ограничение только на output
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        num_beams=4,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return clean_sentence(raw)

def generate_original(text):
    inputs = orig_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = orig_model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"],
                                  max_length=128, no_repeat_ngram_size=3, repetition_penalty=1.2,
                                  num_beams=4, early_stopping=True)
    return orig_tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_byt5(text):
    inputs = byt5_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = byt5_model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"],
                                  max_length=128, no_repeat_ngram_size=3, repetition_penalty=1.2,
                                  num_beams=4, early_stopping=True)
    return byt5_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
examples = [
    {"source": "Я аабажаю шахматы и хачю выиграть.", "target": "Я обожаю шахматы и хочу выиграть."},
    {"source": "Она ушла с уливци нипрощавшись.", "target": "Она ушла с улицы, не прощавшись."},
    {"source": "Привет ета тетрадка вазьми пожалуста.", "target": "Привет, эта тетрадка, возьми пожалуйста."},
    {"source": "Мой брат рисует на стикере и ходит на хокей.", "target": "Мой брат рисует на стикере и ходит на хоккей."}
]

import pandas as pd

rows = []
for ex in examples:
    src = ex["source"]
    tgt = ex["target"]

    row = {
        "Ошибочное": src,
        "Ожидаемое": tgt,
        "Speller": correct_with_speller(src),
        "ByT5": generate_byt5(src),
        "FRED-T5": generate_original(src),
        "FRED-T5 дооб.": generate_finetuned(src)
    }

    rows.append(row)

df = pd.DataFrame(rows)
df.head()

,Ошибочное,Ожидаемое,Speller,ByT5,FRED-T5,FRED-T5 дооб.
0,Я аабажаю шахматы и хачю выиграть.,Я обожаю шахматы и хочу выиграть.,Я обожаю шахматы и хочу выиграть.,абжаю выигрыть шаҳхолотку и перейт на клубы. Х...,Я обожаю шахматы и хочу выиграть. Я хаабабаха ...,Я обожаю шахматы и хочу выиграть.
1,Она ушла с уливци нипрощавшись.,"Она ушла с улицы, не прощавшись.",Она ушла с улицы не попрощавшись.,нипрощавшись. На службу податели в ко мерце з...,Она ушла с улицОно ушла с улицыОна ушлас улицы...,"Она ушла с улицы, непрощавшись."
2,Привет ета тетрадка вазьми пожалуста.,"Привет, эта тетрадка, возьми пожалуйста.",Привет эта тетрадка возьми пожалуйста.,пожалуста\nПривет! Всегда начну с моя купюшка...,Привет эта тетрадка тетрадка возьми возьми пож...,"Привет, это тетрадка, возьми, пожалуйста."
3,Мой брат рисует на стикере и ходит на хокей.,Мой брат рисует на стикере и ходит на хоккей.,Мой брат рисует на стикере и ходит на хоккей.,и ходит на стикере. Моя будущего малыша!\nМы ...,Мой брат рисует на стикере иМой братРисуетМой ...,Мой брат рисует на стикере и ходит на хокей.


In [ ]:
import re

def clean_sentence(text):
    match = re.search(r"(.+?[.!?])", text)
    return match.group(1).strip() if match else text.strip()

def correct_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        num_beams=4,
        early_stopping=True
    )
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return clean_sentence(raw)

In [ ]:
import evaluate
# Загружаем метрики
bleu = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")
rouge = evaluate.load("rouge")

# Предсказания
references = [[ex["target"]] for ex in examples]

pred_speller = [correct_with_speller(ex["source"]) for ex in examples]
pred_byt5    = [generate_byt5(ex["source"]) for ex in examples]
pred_fred    = [generate_original(ex["source"]) for ex in examples]
pred_finetuned = [generate_finetuned(ex["source"]) for ex in examples]

In [ ]:
def collect_metrics(preds, refs):
    rouge_scores = rouge.compute(predictions=preds, references=[r[0] for r in refs])
    return {
        "BLEU": bleu.compute(predictions=preds, references=refs)["score"],
        "chrF": chrf.compute(predictions=preds, references=[r[0] for r in refs])["score"],
    }

metrics_table = pd.DataFrame([
    {"Модель": "Speller", **collect_metrics(pred_speller, references)},
    {"Модель": "ByT5", **collect_metrics(pred_byt5, references)},
    {"Модель": "FRED-T5 (orig)", **collect_metrics(pred_fred, references)},
    {"Модель": "FRED-T5 (дооб.)", **collect_metrics(pred_finetuned, references)},
])

metrics_table.round(2)

,Модель,BLEU,chrF
0,Speller,70.97,89.94
1,ByT5,3.04,30.13
2,FRED-T5 (orig),5.62,24.76
3,FRED-T5 (дооб.),66.87,92.54
